In [12]:
import requests
from bs4 import BeautifulSoup as BS
import json
from tqdm import tqdm
from zipfile import ZipFile, ZIP_DEFLATED
import pandas as pd
import time

import aiohttp, asyncio

In [2]:
proxy_url = '45.81.76.252'
proxy_port = '8000'

proxy = {
    'https': f'http://{proxy_url}:{proxy_port}'
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8'
}

params = {
    'searchString': '',
    'morphology': 'on',
    'search-filter': 'Дате обновления',
    'savedSearchSettingsIdHidden': '',
    'fz94': 'on',
    'fz223': 'on',
    'published': 'on',
    'regarded': 'on',
    'considered': 'on',
    'decisionOnTheComplaintTypeResult': '',
    'receiptDateStart': '',
    'receiptDateEnd': '',
    'updateDateFrom': '',
    'updateDateTo': '',
    'sortBy': 'UPDATE_DATE',
    'pageNumber': '',
    'sortDirection': 'false',
    'recordsPerPage': '_10',
    'showLotsInfoHidden': 'false',
}

url = """

https://zakupki.gov.ru/epz/complaint/search/search_eis.html

""".strip()
host = '/'.join(url.split('/', maxsplit=3)[:3])


In [3]:
def _parse_links_in_page(text: str) -> list[str]:
    # Получам ссылки с каждой страницы (всего 10)
    out_list = list()
    soup = BS(text, 'html.parser')
    blocks_list = soup.find('div', class_ = 'search-registry-entrys-block')
    blocks_list = blocks_list.find_all('div', class_ = 'search-registry-entry-block')

    for block in blocks_list:
        href = block.find('div', class_ = 'd-flex registry-entry__header-mid align-items-center')
        href = host + href.find('span').find('a').get('href')
        out_list.append(href)
    
    return out_list

def _parse_links_in_site() -> list[str]:
    # Проходим по страницам поиска из реестра жалоб
    # всего страниц 100
    # на каждой странице 10 жалоб
    # на выходе получаем список ссылок на страницу с жалобой
    out_list = list()
    
    with tqdm(total=100 ) as pbar:
        for page in range(1,101):
            params['pageNumber'] = f'{page}'
            response = requests.get(url=url,params=params , headers=headers, proxies=proxy)
            out_list.append(_parse_links_in_page(response.text))
            pbar.update(1)
    
    return sum(out_list, [])

In [4]:
# links_list = _parse_links_in_site()

In [5]:
# Сохранение линков в json формате
#--------------------------------------------
# with open('data/links_list.json', 'w', encoding='utf-8') as file:
#     tmp = json.dumps(links_list, indent=4, ensure_ascii=False)
#     file.write(tmp)

In [6]:
# # Загрузка линков из json
# # --------------------------------------------
# with open('data/links_list.json', 'r', encoding='utf-8') as file:
#     tmp = file.read()
#     links_list = json.loads(tmp)

In [28]:
async def _get_pages_async(
                            session: aiohttp.ClientSession,
                            link: str
    ):
    
    async with session.get(url=link, proxy=proxy.get('https', '')) as response:
        response_text = await response.text()
    return response_text

async def _get_pages_tasks(
                            input_links_list: list[str]
    ):
    
    tasks = list()
    async with aiohttp.ClientSession(headers=headers) as session:
        for link in input_links_list:
            task = asyncio.create_task(_get_pages_async(session=session, link=link))
            tasks.append(task)
            
        pages = await asyncio.gather(*tasks)
    return pages

In [ ]:
start = time.time()
test = await _get_pages_tasks(links_list)
print(time.time() -  start)